#Neural Decoding - Progressive additive approach
##Written by Gergana Slaveykova s1070004
##Radboud University- B3 Thesis project

#Imports

In [ ]:
#Mount Google Drive to acess data and storing purposes
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Installs
! pip install mxnet
! pip install nilearn
! pip install nibabel
!pip install git+https://github.com/nipy/nipy.git

In [ ]:
#Imports
import pickle
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


import nibabel as nib
import numpy as np
from nipy.modalities.fmri.experimental_paradigm import load_paradigm_from_csv_file
from nipy.modalities.fmri.design_matrix import make_dmtx
from nipy.labs.viz import plot_map, cm
import matplotlib.pyplot as plt
import pickle

from __future__ import annotations

import os
from types import ModuleType
from typing import Tuple, Union

from nilearn import masking, plotting
from PIL import Image
from scipy.stats import t, zscore
from sklearn.linear_model import RidgeCV

from sklearn.linear_model import LinearRegression
import h5py
import PIL.Image
from scipy import sparse, stats
from numpy.linalg import svd
import time
from scipy import signal

# The combinations code is not optimal. What I did is to enter the combination that I wanted manually. You can in practice do this with function. At the moment while doing the project I found it better for me to have more control on what is going in each combination.

#Main logic

In [ ]:
#If you want to experiment with z-scoring and clipping
def clip_and_z_score(train,test):
  x_tr = np.clip(train, -3, 3)
  x_te_pt = np.clip(test, -3, 3)

  # Z-score normalization
  # Calculate mean and standard deviation on training data
  norm_mean_x = np.mean(x_tr, axis=0)
  norm_std_x = np.std(x_tr, axis=0, ddof=1)
  # Avoid division by zero
  norm_std_x[norm_std_x == 0] = 1

  # Normalization
  x_tr_normalized = (x_tr - norm_mean_x) / norm_std_x
  x_te_pt_normalized = (x_te_pt - norm_mean_x) / norm_std_x

  # Z-scoring the whole dataset
  #x_whole_normalized = (x - np.mean(x, axis=0)) / np.std(x, axis=0, ddof=1)

  return x_tr_normalized,x_te_pt_normalized

In [ ]:
def z_score(x_tr,x_te_pt):
  # Z-score normalization
  # Calculate mean and standard deviation on training data
  norm_mean_x = np.mean(x_tr, axis=0)
  norm_std_x = np.std(x_tr, axis=0, ddof=1)
  # Avoid division by zero
  norm_std_x[norm_std_x == 0] = 1

  # Normalization
  x_tr_normalized = (x_tr - norm_mean_x) / norm_std_x
  x_te_pt_normalized = (x_te_pt - norm_mean_x) / norm_std_x

  # Z-scoring the whole dataset
  #x_whole_normalized = (x - np.mean(x, axis=0)) / np.std(x, axis=0, ddof=1)

  return x_tr_normalized,x_te_pt_normalized

#Code

In [ ]:
#Load latents of the images
w_te= np.load("/content/drive/MyDrive/GOD-NEW-3p/latents/t_te_best.npy")
w_tr= np.load("/content/drive/MyDrive/GOD-NEW-3p/latents/t_tr_best.npy")

In [ ]:
#Load indices of the voxel selection
file_path = "/content/drive/MyDrive/GOD-NEW-3p/Neural-DecodingPartProgresive/indeces_filtered_clipped3.dat"
importantind = np.load(file_path,allow_pickle=True)
for i in importantind:
  print(len(i))


In [ ]:
#Areas of interest.You can comment out given one depenfing on what combination you want
rois = {
        'V1' : 'ROI_V1 = 1',
        'V2' : 'ROI_V2 = 1',
        'V3' : 'ROI_V3 = 1',
        # 'V4' : 'ROI_V4 = 1',
        # 'LOC' : 'ROI_LOC = 1',
        # 'FFA' : 'ROI_FFA = 1',
        # 'PPA' : 'ROI_PPA = 1'
        }

rois.keys()

In [ ]:
#Load brain data
file_path = "/content/drive/MyDrive/GOD-NEW-3p/Neural-DecodingPartProgresive/indeces_filtered_clipped3.dat"
importantind = np.load(file_path,allow_pickle=True)

x_tr = []
x_te = []
#iterate over all regions
for roi_index, roi in enumerate(rois):
    _roi = np.load(f"/content/drive/MyDrive/GOD-NEW-3p/my_experiment_{roi}_hyperaligned.npy")  # load in roi, all voxels, you want to select a mask
    tr=_roi[:1200,importantind[roi_index]] #filter the voxels
    te=_roi[1200:1250,importantind[roi_index]]
    #if you want you can z-score
    #tr,te= z_score(tr,te)
    #tr,te= clip_and_z_score(tr,te)
    x_tr.append(tr)
    x_te.append(te)

print(len(x_te))

In [ ]:
#Combine the regions together so they can be given to the linear model easly
concat_te = np.concatenate(x_te, axis=1)
print(concat_te.shape)

concat_tr = np.concatenate(x_tr, axis=1)
print(concat_tr.shape)

In [ ]:
#train linear model on the combined filtered braind data
modelWhole = LinearRegression().fit(concat_tr, w_tr)
#predicted latents
y_teWhole = modelWhole.predict(concat_te)

print(y_teWhole.shape)

#choose directory
file_path="/content/drive/MyDrive/filtered_my_method/"

#pick the appropriate name for the file and store it
with open(f'{file_path}v3v2v1_y_te.npy', 'wb') as fp:
            pickle.dump(y_teWhole, fp)

(50, 512)
